In [2]:
# This is the pysentimiento calculations file
# pip install pysentimiento
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="irony", lang="en")

c:\Users\Jonas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\modeling_utils.py:367: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(

In [6]:
# Test cell to test how it works
result = analyzer.predict('Wow... soooo cool')
type(result.probas['ironic'])
print(result)

AnalyzerOutput(output=ironic, probas={ironic: 0.971, not ironic: 0.029})


In [6]:
def calcFScore(truepos, trueneg, falsepos, falseneg):
    FScoreResults = {}
    FScoreResults['accuracy'] = (truepos + trueneg)/(truepos + trueneg + falsepos + falseneg)
    FScoreResults['accuracy'] = (truepos + trueneg)/(truepos + trueneg + falsepos + falseneg)
    if (truepos + falsepos > 0):
        FScoreResults['precision'] = truepos/(truepos + falsepos)
    else:
        FScoreResults['precision'] = 0

    if(truepos + falseneg > 0):
        FScoreResults['recall'] = truepos/(truepos + falseneg)
    else:
        FScoreResults['recall'] = 0
        
    if((FScoreResults['precision'] + FScoreResults['recall']) > 0):
        FScoreResults['F1'] = (2 * FScoreResults['precision'] * FScoreResults['recall'])/(FScoreResults['precision'] + FScoreResults['recall'])
    else:
        FScoreResults['F1'] = 0
    return FScoreResults

In [7]:
# Calculate the scores based on the answers from pysentimiento, this method is simply adapted from the other scores methods for GPT
def scoresSentimiento(dataset, resultColumn):
    truepos = 0
    falsepos = 0
    trueneg = 0
    falseneg = 0

    for index, row in dataset.iterrows():
        sentimientoEval = row[resultColumn]
        if (int(row[1]) == 1):
            if (int(sentimientoEval) == 1):
                truepos = truepos + 1
            else:
                falseneg = falseneg + 1
        elif (int(row[1]) == 0):
            if (int(sentimientoEval) == 1):
                falsepos = falsepos + 1
            else:
                trueneg = trueneg + 1

    numResults = calcFScore(truepos, trueneg, falsepos, falseneg)
    numResults['tp'] = truepos
    numResults['fp'] = falsepos
    numResults['fn'] = falseneg
    numResults['tn'] = trueneg
    return numResults

In [8]:
# converts responses to 1 or 0 classification with threshold 0.5
import pandas as pd

datasetName = "manual_select_odd"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = analyzer.predict(row[0])
    if(result.probas['ironic'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores

{'accuracy': 0.79,
 'precision': 0.7592592592592593,
 'recall': 0.8367346938775511,
 'F1': 0.796116504854369,
 'tp': 41,
 'fp': 13,
 'fn': 8,
 'tn': 38}

In [14]:
# MAIN CELL does the analysis and creates the metadata file for this run type
import pandas as pd
import os

datasetName = "manual_select_odd"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = analyzer.predict(row[0])
    if(result.probas['ironic'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores

linesToWrite = ['Results for Pysentimiento using dataset ' + datasetName +  ': \n',
        'Confusion Matrix:' + '\n',
        str(scores['tp']) + '  ' + str(scores['fn']) + '\n',
        str(scores['fp']) + '  ' + str(scores['tn']) + '\n',
        'Accuracy: ' + str(scores['accuracy']) + '\n',
        'Precision: ' + str(scores['precision']) + '\n',
        'Recall: ' + str(scores['recall']) + '\n',
        'F1-Score: ' + str(scores['F1']) + '\n\n']

path = "Other LLMs\\pysentimiento results\\" + datasetName
if not os.path.exists(path):
    os.makedirs(path)

file = open(path + "\\metadata.txt", "w")

data.to_csv(path + "\\results.csv", index=False)

file.writelines(linesToWrite)
file.close()